# Overview of MNE-Python's data structures

In [2]:
import mne
import os.path as op
from __future__ import print_function
import numpy as np

## Creating `Info` objects

Normally, `Info` objects are created by the various [data import functions](http://martinos.org/mne/dev/python_reference.html#reading-raw-data). However, if you wish to create one from scratch, you can use the [`create_info`](http://martinos.org/mne/stable/generated/mne.create_info.html#mne.create_info) function to initialize the minimally required fields. Further fields can be assigned later as one would with a regular dictionary.

The following creates the absolute minimum info structure:

In [18]:
n_channels = 32
sampling_rate = 200
info = mne.create_info(32, sampling_rate)
print(info)

<Info | 13 non-empty fields
    bads : list | 0 items
    ch_names : list | 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
    chs : list | 32 items (MISC: 32)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : dict | 3 items
    events : list | 0 items
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    meas_date : numpy.ndarray | 1970-01-01 01:00:00
    nchan : int | 32
    projs : list | 0 items
    sfreq : float | 200.0
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    filename : NoneType
    highpass : NoneType
    hpi_subsystem : NoneType
    line_freq : NoneType
    lowpass : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
>


More extended example:

In [19]:
# Names for each channel
channel_names = ['MEG1', 'MEG2', 'Cz', 'Pz', 'EOG']

# The type (mag, grad, eeg, eog, misc, ...) of each channel
channel_types = ['grad', 'grad', 'eeg', 'eeg', 'eog']

# The sampling rate of the recording
sfreq = 1000  # in Hertz

# The EEG channels use the standard naming strategy.
# By supplying the 'montage' parameter, approximate locations
# will be added for them 
montage = 'standard_1005'

# Initialize required fields
info = mne.create_info(channel_names, sfreq, channel_types, montage)

# Add some more information
info['description'] = 'My custom dataset'
info['bads'] = ['Pz']  # Names of bad channels

print(info)

<Info | 14 non-empty fields
    bads : list | Pz
    ch_names : list | MEG1, MEG2, Cz, Pz, EOG
    chs : list | 5 items (EOG: 1, EEG: 2, GRAD: 2)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 17 items
    dev_head_t : dict | 3 items
    events : list | 0 items
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    meas_date : numpy.ndarray | 1970-01-01 01:00:00
    nchan : int | 5
    projs : list | 0 items
    sfreq : float | 1000.0
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    filename : NoneType
    highpass : NoneType
    hpi_subsystem : NoneType
    line_freq : NoneType
    lowpass : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
>


**NOTE:** When assigning new values to the fields of an `Info` object, it is important that the fields are consistent:

 - The length of the channel information field `chs` must be `nchan`.
 - The length of the `ch_names` field must be `nchan`.
 - The `ch_names` field should be consistent with the `name` field of the channel information contained in `chs`.

In [18]:
# List all information about the first data channel
print(raw.info['chs'][0])

{'loc': array([-0.1066    ,  0.0464    , -0.0604    , -0.0127    ,  0.0057    ,
       -0.99990302, -0.186801  , -0.98240298, -0.0033    , -0.98232698,
        0.18674099,  0.013541  ], dtype=float32), 'kind': 1, 'unit_mul': 0, 'coil_trans': array([[-0.0127    , -0.186801  , -0.98232698, -0.1066    ],
       [ 0.0057    , -0.98240298,  0.18674099,  0.0464    ],
       [-0.99990302, -0.0033    ,  0.013541  , -0.0604    ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]]), 'ch_name': 'MEG 0113', 'coil_type': 3012, 'coord_frame': 1, 'logno': 113, 'cal': 3.1600000394149674e-09, 'eeg_loc': None, 'range': 0.00030517578125, 'scanno': 1, 'unit': 201}


### Creating `Raw` objects

To create a `Raw` object from scratch, you can use the `RawArray` class, which implements raw data that is backed by a numpy array. Its constructor simply takes the data matrix and `Info` object:

In [3]:
# Generate some random data
data = np.random.randn(5, 1000)

# Initialize an info structure
info = mne.create_info(
    ch_names=['MEG1', 'MEG2', 'EEG1', 'EEG2', 'EOG'],
    ch_types=['grad', 'grad', 'eeg', 'eeg', 'eog'],
    sfreq=100
)

custom_raw = mne.io.RawArray(data, info)
print(custom_raw)

Creating RawArray with float64 data, n_channels=5, n_times=1000
    Range : 0 ... 999 =      0.000 ...     9.990 secs
Ready.
<RawArray  |  n_channels x n_times : 5 x 1000>


## Creating `Epochs` objects

To create an `Epochs` object from scratch, you can use the `EpochsArray` class, which uses a numpy array directly without wrapping a raw object.

In [4]:
# Generate some random data: 10 epochs, 5 channels, 111 samples per epoch
data = np.random.randn(10, 5, 111)

# Initialize an info structure
info = mne.create_info(
    ch_names=['MEG1', 'MEG2', 'EEG1', 'EEG2', 'EOG'],
    ch_types=['grad', 'grad', 'eeg', 'eeg', 'eog'],
    sfreq=100
)

# Create an event matrix: 10 events with a duration of 1 sample, alternating event codes
events = np.array([
    [0, 1, 1],
    [1, 1, 2],
    [2, 1, 1],
    [3, 1, 2],       
    [4, 1, 1],
    [5, 1, 2],
    [6, 1, 1],
    [7, 1, 2],
    [8, 1, 1],
    [9, 1, 2],
])

# More information about the event codes: subject was either smiling or frowning
event_id = dict(smiling=1, frowning=2)

# Trials were cut from -0.1 to 1.0 seconds
tmin = -0.1

# Create epochs object
custom_epochs = mne.EpochsArray(data, info, events, tmin, event_id)
print(custom_epochs)

10 matching events found
Adding average EEG reference projection.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
No baseline correction applied...
0 bad epochs dropped
<EpochsArray  |  n_events : 10 (all good), tmin : -0.1 (s), tmax : 1.0 (s), baseline : None,
 'frowning': 5, 'smiling': 5>
